# SciPy solve_ivp example for solving 2D diffusion equation in a rectangular grid

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import ipywidgets as widgets
%matplotlib widget

## Modelled system

The modelled system is a rectangular surface of a pond. When a contaminant is spilled in the pond it will diffuse on the surface. The evolution of the contaminant concentration C is described by Fick's second law

\begin{align*}
    \frac{\partial C}{\partial t} &= D \Delta^2 C \\
    &= D \left( \frac{\partial^2 C}{\partial x^2} + \frac{\partial^2 C}{\partial y^2} \right)
\end{align*}

where D is the diffusion coefficient of the contaminant in water

We well be using the [method of lines](http://hplgit.github.io/prog4comp/doc/pub/p4c-sphinx-Python/._pylight006.html#reduction-of-a-pde-to-a-system-of-odes) approach to transform the PDE into a system of ODEs. This is done by discretizing the spatial dimension.

We will follow the central difference scheme in the above link. The change of concentration in an element i is

\begin{equation*}
    \frac{\mathrm{d}C_{i,j}}{\mathrm{d}t} = D\left( \frac{C(x_{i+1},t) - 2C(x,t) + C(x_{i-1},t)}{(\Delta x)^2} +\frac{C(y_{i+1},t) - 2C(y,t) + C(y_{i-1},t)}{(\Delta y)^2} \right)
\end{equation*}

Finally, we need to decide what is the change of concentration at the boundaries of the rectangle (boundary conditions). For simplicity we will assume Dirichlet boundary condition and assume that the concentration at the boundary is 

\begin{equation*}
    C_{\mathrm{boundary}} = C_0
\end{equation*}

Now that we have discretized the equation we are left with $(N-1) \cdot (M-1)$ ordinary differential equations of the form

\begin{align*}
\frac{\mathrm{d}C_{1,1}}{\mathrm{d}t} &= D\left(  \frac{C_{2,1} - 2C_{1, 1} + C_{0, 1},t)}{(\Delta x)^2} +\frac{C_{1,2} - 2C_{1, 1} + C_{1,0}}{(\Delta y)^2} \right) \\
&\;\;\vdots \\
\frac{\mathrm{d}C_{N-1,M-1}}{\mathrm{d}t} &= D\left(  \frac{C_{N,M-1} - 2C_{N-1, M-1} + C_{N-2, M},t)}{(\Delta x)^2} +\frac{C_{N-1,M} - 2C_{N-1, M-1} + C_{N-1, M-2}}{(\Delta y)^2} \right)
\end{align*}

where N is the number of horizontal elements in the rectangular grid and M is the number of vertical elements